In [ ]:
import os
from dotenv import load_dotenv
import pyodbc
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine

# Load environment variables
load_dotenv()

# Read values
server = os.getenv("DB_SERVER")
port = os.getenv("DB_PORT", "1433")  # Default SQL Server port
database = os.getenv("DB_NAME")
username = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")

# Create connection string
connection_string = f"mssql+pyodbc://{username}:{password}@{server}:{port}/{database}?driver=ODBC+Driver+17+for+SQL+Server"

# Connect
conn = create_engine(connection_string)

# Define load_query function to load SQL queries from files
def load_query(name):
	path = Path("../scripts") / f"{name}.sql"
	if not path.exists():
		raise FileNotFoundError(f"Query file {path} does not exist.")
	return path.read_text()

# Load Dataset from SQL Database

In [7]:
# Load contracts data
contracts_df = pd.read_sql(load_query('get_contracts'), conn)
contracts_df


,contract_id,supplier_id,contract_value_million,contract_type,award_date,expected_duration_months,status
0,CTR00000,SUP0056,1.75,R&D,2025-03-30,37,delayed
1,CTR00001,SUP0087,6.18,Tech Services,2021-07-14,25,terminated
2,CTR00002,SUP0224,2.95,R&D,2024-12-04,15,terminated
3,CTR00003,SUP0104,0.23,Production,2022-06-15,34,terminated
4,CTR00004,SUP0252,0.13,Production,2024-05-14,46,delayed
...,...,...,...,...,...,...,...
4995,CTR04995,SUP0030,8.05,R&D,2022-11-04,11,completed
4996,CTR04996,SUP0296,3.46,Production,2023-12-18,14,completed
4997,CTR04997,SUP0095,2.02,Production,2020-06-03,26,completed
4998,CTR04998,SUP0270,0.14,Sustainment,2020-07-18,17,delayed


In [8]:
# Load suppliers data
suppliers_df = pd.read_sql(load_query('get_suppliers'), conn)
suppliers_df

,supplier_id,supplier_name,past_contracts,average_delay_days,compliance_issues
0,SUP0003,Wiley-Moore,19,1,1
1,SUP0005,"Liu, Jones and Jones",17,142,3
2,SUP0006,Page LLC,30,163,1
3,SUP0008,Potter PLC,29,8,0
4,SUP0011,Rogers and Sons,28,92,2
...,...,...,...,...,...
310,SUP0492,Garza-Hamilton,33,68,1
311,SUP0497,Murphy Group,7,35,2
312,SUP0498,"Pierce, Willis and Melendez",4,45,4
313,SUP0499,Bush and Sons,16,110,3


In [9]:
# Load delays data
delays_df = pd.read_sql(load_query('get_delays'), conn)
delays_df

,contract_id,supplier_id,contract_value_million,contract_type,award_date,expected_duration_months,status,delay_days
0,CTR00000,SUP0056,1.75,R&D,2025-03-30,37,delayed,153
1,CTR00004,SUP0252,0.13,Production,2024-05-14,46,delayed,99
2,CTR00005,SUP0374,0.50,Sustainment,2021-12-28,57,delayed,99
3,CTR00012,SUP0116,3.08,R&D,2022-07-17,12,delayed,149
4,CTR00013,SUP0395,4.53,Production,2021-05-03,11,delayed,31
...,...,...,...,...,...,...,...,...
1264,CTR04977,SUP0344,2.77,R&D,2024-12-30,43,delayed,104
1265,CTR04989,SUP0194,1.24,Production,2020-09-23,51,delayed,223
1266,CTR04992,SUP0222,1.27,Production,2021-01-20,33,delayed,121
1267,CTR04994,SUP0337,1.26,Tech Services,2023-02-07,41,delayed,194


# Exploratory Data Analysis (EDA)

In [20]:
# Dataset Overview
print("Contracts Data Overview:")
contracts_df.info()
print("\nSuppliers Data Overview:")
suppliers_df.info()
print("\nDelays Data Overview:")
delays_df.info()

Contracts Data Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   contract_id               5000 non-null   object 
 1   supplier_id               5000 non-null   object 
 2   contract_value_million    5000 non-null   float64
 3   contract_type             5000 non-null   object 
 4   award_date                5000 non-null   object 
 5   expected_duration_months  5000 non-null   int64  
 6   status                    5000 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 273.6+ KB

Suppliers Data Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   supplier_id         315 non-null    object
 1   supplier_name       315 non-nul